Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [16]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [17]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [18]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    # 3x2x3
    # array=[
    #        [
    #         [1,2,3],
    #         [4,5,6]
    #        ],
    #        [
    #         [7,8,9],
    #         [10,11,12]
    #        ],
    #        [
    #         [13,14,15]
    #         [16,17,18]
    #        ],
    #       ]
    # after array.reshape(-1, 2 * 3)
    # reslut=[[1,2,3,4,5,6],
    #         [7,8,9,10,11,12]
    #         [13,14,15,16,17,18]
    #        ]
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    # np.arrange(10): category = [0,1,2,3,4,5,6,7,8,9]
    # category == 4 => [False, False, False, False,  True, False, False, False, False, False]
    # (category == 4).astype(np.float32) => [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [19]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression with l2 loss function
-----------------------------------------

In [20]:
# regularizing with beta = 0.01

# multinomial logistic regression
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 801

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])

        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels) )

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 46.506275
Training accuracy: 9.5%
Validation accuracy: 13.3%
Loss at step 100: 11.777938
Training accuracy: 74.1%
Validation accuracy: 73.1%
Loss at step 200: 4.478943
Training accuracy: 79.3%
Validation accuracy: 77.6%
Loss at step 300: 1.983642
Training accuracy: 82.4%
Validation accuracy: 80.3%
Loss at step 400: 1.133908
Training accuracy: 83.7%
Validation accuracy: 81.5%
Loss at step 500: 0.841569
Training accuracy: 84.0%
Validation accuracy: 82.0%
Loss at step 600: 0.739828
Training accuracy: 84.2%
Validation accuracy: 82.3%
Loss at step 700: 0.704094
Training accuracy: 84.4%
Validation accuracy: 82.3%
Loss at step 800: 0.691447
Training accuracy: 84.4%
Validation accuracy: 82.4%
Test accuracy: 89.1%


Neural network wiht l2 loss function
------------------------------------

In [22]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.935638
Minibatch accuracy: 8.6%
Validation accuracy: 14.0%
Minibatch loss at step 500: 0.866493
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.671214
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.931760
Minibatch accuracy: 74.2%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 0.801875
Minibatch accuracy: 82.8%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 0.615229
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.746723
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Test accuracy: 88.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Log reg with l2 loss and small training data
--------------------------------------------

In [27]:
# multinomial logistic regression 

train_subset = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [28]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])

        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels) )

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    
# Training accuracy goes to 100% due to the small sample size, 
# however, the model is not as good the validation samples.

Initialized
Loss at step 0: 50.424183
Training accuracy: 10.0%
Validation accuracy: 11.0%
Loss at step 100: 11.088320
Training accuracy: 100.0%
Validation accuracy: 45.4%
Loss at step 200: 4.136235
Training accuracy: 100.0%
Validation accuracy: 52.4%
Loss at step 300: 1.596785
Training accuracy: 100.0%
Validation accuracy: 59.5%
Loss at step 400: 0.671136
Training accuracy: 100.0%
Validation accuracy: 63.9%
Loss at step 500: 0.333710
Training accuracy: 100.0%
Validation accuracy: 66.3%
Loss at step 600: 0.210482
Training accuracy: 100.0%
Validation accuracy: 67.4%
Loss at step 700: 0.165306
Training accuracy: 100.0%
Validation accuracy: 68.1%
Loss at step 800: 0.148624
Training accuracy: 100.0%
Validation accuracy: 68.3%
Test accuracy: 75.0%


Neural network with l2 loss and small training data
---------------------------------------------------

In [29]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [30]:
num_steps = 3001

train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

# We see overfitting occuring on neural network model when the training set is 3 times smaller.

Initialized
Minibatch loss at step 0: 51.832253
Minibatch accuracy: 6.2%
Validation accuracy: 12.8%
Minibatch loss at step 500: 0.575074
Minibatch accuracy: 96.9%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 0.339519
Minibatch accuracy: 99.2%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 0.286044
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.270461
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 2500: 0.262154
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 3000: 0.266884
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Test accuracy: 83.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [31]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 457.210358
Minibatch accuracy: 8.6%
Validation accuracy: 32.6%
Minibatch loss at step 500: 32.717113
Minibatch accuracy: 74.2%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 14.430128
Minibatch accuracy: 74.2%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 11.899036
Minibatch accuracy: 68.8%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 11.126209
Minibatch accuracy: 74.2%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 3.144731
Minibatch accuracy: 75.8%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 5.168482
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Test accuracy: 86.7%


In [33]:
num_steps = 3001

train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 515.042542
Minibatch accuracy: 8.6%
Validation accuracy: 25.9%
Minibatch loss at step 500: 3.100446
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 1.869233
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 1.163270
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 0.742715
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.488512
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 0.347113
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 84.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Trying smaller beta and bigger num steps
----------------------------------------

In [37]:
batch_size = 128
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [38]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 490.814728
Minibatch accuracy: 10.9%
Validation accuracy: 29.9%
Minibatch loss at step 500: 43.036697
Minibatch accuracy: 76.6%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 20.025515
Minibatch accuracy: 78.1%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 11.937448
Minibatch accuracy: 64.1%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 11.924856
Minibatch accuracy: 73.4%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 4.176146
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 3.977453
Minibatch accuracy: 77.3%
Validation accuracy: 79.9%
Minibatch loss at step 3500: 8.576484
Minibatch accuracy: 71.1%
Validation accuracy: 79.9%
Minibatch loss at step 4000: 7.854161
Minibatch accuracy: 77.3%
Validation accuracy: 80.7%
Minibatch loss at step 4500: 9.309613
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Minibatch loss at step 5000: 0.971546
Minibatch accuracy: 80.5%
Validation a

## Trying keep prob of 0.1

In [39]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [40]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.1}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1133.790283
Minibatch accuracy: 10.2%
Validation accuracy: 33.3%
Minibatch loss at step 500: 457087.625000
Minibatch accuracy: 48.4%
Validation accuracy: 69.5%
Minibatch loss at step 1000: 14255584.000000
Minibatch accuracy: 48.4%
Validation accuracy: 72.0%
Minibatch loss at step 1500: 244643632.000000
Minibatch accuracy: 36.7%
Validation accuracy: 69.1%
Minibatch loss at step 2000: 22856202240.000000
Minibatch accuracy: 44.5%
Validation accuracy: 70.1%
Minibatch loss at step 2500: 264826830848.000000
Minibatch accuracy: 45.3%
Validation accuracy: 71.9%
Minibatch loss at step 3000: 12656340631552.000000
Minibatch accuracy: 43.0%
Validation accuracy: 66.0%
Test accuracy: 72.1%


## Trying with learning rate and multiple keep probs

In [41]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [43]:
num_steps = 3001

for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    print(">>with keep prob of " + str(kp))
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


>>with keep prob of 0.5
Initialized
Minibatch loss at step 0: 518.856323
Minibatch accuracy: 12.5%
Validation accuracy: 31.6%
Minibatch loss at step 500: 38.629776
Minibatch accuracy: 77.3%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 16.603941
Minibatch accuracy: 71.9%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 11.881701
Minibatch accuracy: 63.3%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 13.558303
Minibatch accuracy: 72.7%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 3.389016
Minibatch accuracy: 84.4%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 8.321461
Minibatch accuracy: 72.7%
Validation accuracy: 79.9%
Test accuracy: 86.7%
>>with keep prob of 0.6
Initialized
Minibatch loss at step 0: 449.093201
Minibatch accuracy: 10.2%
Validation accuracy: 30.4%
Minibatch loss at step 500: 14.470038
Minibatch accuracy: 77.3%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 24.393856
Minibatch accuracy: 71.1%
Validation accuracy: 7

## Trying different learning rates

In [44]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [45]:
for lr in np.arange(0.0001, 0.001, 0.0001).tolist():

    print('>>trying lr of ' + str(lr))
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, learnr : lr}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

>>trying lr of 0.0001
Initialized
Minibatch loss at step 0: 385.842987
Minibatch accuracy: 11.7%
Validation accuracy: 40.5%
Minibatch loss at step 500: 11.926297
Minibatch accuracy: 77.3%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 13.397211
Minibatch accuracy: 77.3%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 8.116122
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 11.611623
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 1.583406
Minibatch accuracy: 86.7%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 2.704599
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.8%
>>trying lr of 0.0002
Initialized
Minibatch loss at step 0: 374.371277
Minibatch accuracy: 8.6%
Validation accuracy: 33.7%
Minibatch loss at step 500: 13.115983
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 18.906265
Minibatch accuracy: 74.2%
Validation accuracy: 81.0%
M

## Trying 2 layer neural network

In [46]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu(tf.matmul( test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)

In [47]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, learnr : 0.001}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict )
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6134.153809
Minibatch accuracy: 8.6%
Validation accuracy: 10.5%
Minibatch loss at step 500: 307.032623
Minibatch accuracy: 77.3%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 217.596283
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 342.984344
Minibatch accuracy: 70.3%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 238.399216
Minibatch accuracy: 78.9%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 88.152672
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 115.266457
Minibatch accuracy: 85.9%
Validation accuracy: 80.6%
Test accuracy: 87.8%


In [48]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu(tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu(tf.matmul( test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)

In [51]:
num_steps = 3001
scores = {}
for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    for lr in np.arange(0.0001, 0.001, 0.0001).tolist():
        print(">> with keep prob of " + str(kp))
        print(">> with lr  " + str(lr))
        with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict )
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)
print(scores)

>> with keep prob of 0.5
>> with lr  0.0001
Initialized
Minibatch loss at step 0: 9798.194336
Minibatch accuracy: 9.4%
Validation accuracy: 7.5%
Minibatch loss at step 500: 2180.100342
Minibatch accuracy: 49.2%
Validation accuracy: 71.1%
Minibatch loss at step 1000: 1889.402710
Minibatch accuracy: 51.6%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 2102.829346
Minibatch accuracy: 52.3%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 1606.990356
Minibatch accuracy: 56.2%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 1101.307617
Minibatch accuracy: 64.1%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 1144.654663
Minibatch accuracy: 65.6%
Validation accuracy: 79.1%
Test accuracy: 86.5%
>> with keep prob of 0.5
>> with lr  0.0002
Initialized
Minibatch loss at step 0: 10161.313477
Minibatch accuracy: 8.6%
Validation accuracy: 7.4%
Minibatch loss at step 500: 2134.608643
Minibatch accuracy: 46.1%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 143

## Trying 2 NN with different loss function

In [52]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * ( tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_weights2) ))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu(tf.matmul(test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)

In [53]:
num_steps = 3001
scores = {}
for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    for lr in np.arange(0.0001, 0.001, 0.0001).tolist():
        print(">> with keep prob of " + str(kp))
        print(">> with lr  " + str(lr))
        with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict )
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)
print(scores)

>> with keep prob of 0.5
>> with lr  0.0001
Initialized
Minibatch loss at step 0: 9403.234375
Minibatch accuracy: 9.4%
Validation accuracy: 10.2%
Minibatch loss at step 500: 3556.572266
Minibatch accuracy: 39.8%
Validation accuracy: 71.3%
Minibatch loss at step 1000: 2401.669434
Minibatch accuracy: 50.0%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 2176.594238
Minibatch accuracy: 53.9%
Validation accuracy: 77.9%
Minibatch loss at step 2000: 2425.125488
Minibatch accuracy: 56.2%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 1625.495605
Minibatch accuracy: 62.5%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 1798.953857
Minibatch accuracy: 58.6%
Validation accuracy: 79.5%
Test accuracy: 86.5%
>> with keep prob of 0.5
>> with lr  0.0002
Initialized
Minibatch loss at step 0: 10078.695312
Minibatch accuracy: 13.3%
Validation accuracy: 13.3%
Minibatch loss at step 500: 3001.082764
Minibatch accuracy: 50.8%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 

## Trying NN with 5 layers

In [54]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512
hidden_nodes3 = 256
hidden_nodes4 = 128
hidden_nodes5 = 64

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu(tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer 2
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # new hidden layer 3
    hidden_weights3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3]) )
    hidden_biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
    hidden_layer3 = tf.nn.relu(tf.matmul(hidden_layer_drop2, hidden_weights3) + hidden_biases3)
    
    # add dropout on hidden layer 3
    hidden_layer_drop3 = tf.nn.dropout(hidden_layer3, keep_prob)
    
    # new hidden layer 4
    hidden_weights4 = tf.Variable(tf.truncated_normal([hidden_nodes3, hidden_nodes4]) )
    hidden_biases4 = tf.Variable(tf.zeros([hidden_nodes4]))
    hidden_layer4 = tf.nn.relu(tf.matmul(hidden_layer_drop3, hidden_weights4) + hidden_biases4)
    
    # add dropout on hidden layer 4
    hidden_layer_drop4 = tf.nn.dropout(hidden_layer4, keep_prob)
    
    # new hidden layer 5
    hidden_weights5 = tf.Variable(tf.truncated_normal([hidden_nodes4, hidden_nodes5]) )
    hidden_biases5 = tf.Variable(tf.zeros([hidden_nodes5]))
    hidden_layer5 = tf.nn.relu(tf.matmul( hidden_layer_drop4, hidden_weights5) + hidden_biases5)
    
    # add dropout on hidden layer 5
    hidden_layer_drop5 = tf.nn.dropout(hidden_layer5, keep_prob)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes5, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop5, weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)  
    valid_relu3 = tf.nn.relu(tf.matmul(valid_relu2, hidden_weights3) + hidden_biases3)   
    valid_relu4 = tf.nn.relu(tf.matmul(valid_relu3, hidden_weights4) + hidden_biases4)
    valid_relu5 = tf.nn.relu(tf.matmul(valid_relu4, hidden_weights5) + hidden_biases5)   
    
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu5, weights) + biases) 
    
    test_relu1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu(tf.matmul(test_relu1, hidden_weights2) + hidden_biases2)   
    test_relu3 = tf.nn.relu(tf.matmul(test_relu2, hidden_weights3) + hidden_biases3)  
    test_relu4 = tf.nn.relu(tf.matmul(test_relu3, hidden_weights4) + hidden_biases4)   
    test_relu5 = tf.nn.relu(tf.matmul(test_relu4, hidden_weights5) + hidden_biases5)  
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu5, weights) + biases)

In [55]:
num_steps = 3001
scores = {}
for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    for lr in np.arange(0.0001, 0.001, 0.0001).tolist():
        print(">> with keep prob of " + str(kp))
        print(">> with lr  " + str(lr))
        with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict )
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)
print(scores)

>> with keep prob of 0.5
>> with lr  0.0001
Initialized
Minibatch loss at step 0: 8755446.000000
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: nan
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 16.4%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%
>> with keep prob of 0.5
>> with lr  0.0002
Initialized
Minibatch loss at step 0: 10600961.000000
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 500: 11362382575572250263552.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 11360318801043007733760.0